[View in Colaboratory](https://colab.research.google.com/github/adityasingh1993/100DayMLChallenge/blob/master/NLP_Bigram.ipynb)

In [1]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2018-10-24 15:12:45--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2018-10-24 15:12:45--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  2.52MB/s    in 9m 44s  

2018-10-24 15:22:29 (3.06 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



In [2]:
!unzip glove.42B.300d.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
print("Loading Word Vector")
word2vec={}
idx2word=[]
embedding=[]

with open('glove.42B.300d.txt') as f:
  for line in f:
    values=line.split()
    word=values[0]
    vec=np.asarray(values[1:],dtype='float32')
    word2vec[word]=vec
    embedding.append(vec)
    idx2word.append(word)
    
  print("found {} number of words".format(len(word2vec)))
  embedding=np.asarray(embedding)
  V,D=embedding.shape

Loading Word Vector
found 1917494 number of words


In [2]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [0]:
from nltk.corpus import brown
import operator


In [0]:
keep_word=set(['king','man','queen','woman','india','england'])

In [0]:
def getSentences():
  return brown.sents()

def getSentence_Word2idx():
  i=2
  sentences=getSentences()
  indexed_sentences=[]
  word2idx_brown={'START':0,'END':1}
  for sentence in sentences:
    indexed_sentence=[]
    for token in sentence:
      token=token.lower()
      if token not in word2idx_brown:
        word2idx_brown[token]=i
        i+=1
      indexed_sentence.append(word2idx_brown[token])
    indexed_sentences.append(indexed_sentence)
  
  print("{} is th size of vocab".format(i))
  print(len(word2idx_brown))
  
  return indexed_sentences,word2idx_brown
  
 

In [0]:
def getSentence_Word2idx_liitVocab(n_vocabs=2000,keep_word=keep_word):
  i=2
  sentences=getSentences()
  indexed_sentences=[]
  word2idx_brown={'START':0,'END':1}
  idx2word_brown=['START','END']
  
  word_idx_count={
      0:float('inf'),
      1:float('inf')
  }
  
  for sentence in sentences:
    indexed_sentence=[]
    for token in sentence:
      token=token.lower()
      if token not in word2idx_brown:
        idx2word_brown.append(token)
        word2idx_brown[token]=i
        i+=1
        indexed_sentence.append(word2idx_brown[token])
      idx=word2idx_brown[token]
      word_idx_count[idx]=word_idx_count.get(idx,0)+1
    indexed_sentences.append(indexed_sentence)
      
      
      
  for word in keep_word:
    word_idx_count[word2idx_brown[word]]=float('inf')
    
  sorted_word_idx_count=sorted(word_idx_count.items(),key=operator.itemgetter(1),reverse=True)
  word2idx_small={}
  word2idx_small={'START':0,'END':1}
  new_idx=0
  idx_new_idx_map={}
  
  for idx,count in sorted_word_idx_count:
    word=idx2word_brown[idx]
    word2idx_small[word]=idx
    idx_new_idx_map[idx]=new_idx
    new_idx+=1
    
 
  word2idx_small['UNKNOWN']=new_idx
  unknown=new_idx
  
  assert('START' in word2idx_small)
  assert('END' in word2idx_small)
  assert('UNKNOWN' in word2idx_small)
  sentence_small=[]
  for sentence in indexed_sentences:
    if len(sentence)>1:
      new_sentence=[idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence ]
      sentence_small.append(new_sentence)
      
      
  return sentence_small,word2idx_small,idx_new_idx_map
      
      
     

In [0]:
def get_bigram_probs(sentences,V,start_idx,end_idx,smoothing=1):
  bigram_probs=np.ones((V,V))*smoothing
  for sentence in sentences:
     for i in range(len(sentence)):
        if i==0:
          bigram_probs[start_idx,sentence[i]]+=1
        else:
          bigram_probs[sentence[i-1],sentence[i]]+=1
          
        if i==len(sentence)-1:
          bigram_probs[sentence[i-1],end_edx]+=1
    
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
          
  return bigram_probs
          
        
          

In [0]:
def get_Score(sentences,start_idx,end_idx):
  score=0
  for sentence in sentences:
    for i in range(len(ssentence)):
      if i==0:
        score +=np.log(bigram_probs[start_idx,sentence[i]])
      else:
         score += np.log(bigram_probs[sentence[i-1],sentence[i]])
        
      if i==len(sentence)-1:
          score += np.log(bigram_probs[sentence[i-1],end_edx])
          
    return score/len(sentence)+1
  

In [0]:
def main():
  sentence_small,word2idx_small,idx2word_small=getSentence_Word2idx_liitVocab(1000)
  V=len(word2idx_small)
  print(len(word2idx_small))
  start_idx=word2idx_small['START']
  end_idx=word2idx_small['END']
  bigram_probs=get_bigram_probs(sentence_small,V,start_idx,end_idx,smoothing=1)
  return bigram_probs,word2idx_small

In [0]:
bigram_probs,word2idx_small=main()

49818


In [0]:
idx2word_small=dict((k,v) for k,v in iteritems(word2idx_small))

In [0]:
def get_words(sentence):
  return ' '.join( idx2word_small[i] in sentence)

In [0]:
sample_prob=np.ones((V))
sample_prob[start_idx]=0
sample_prob[end_idx]=0
sample_prob /= sample_prob.sum()

In [0]:
while True:
  real_idx=np.random.choice(len(sentences))
  real=sentences[real_idx]
  fake=np.random.choices(V,size=len(real),p=sample_prob)
  print("Real is {0} and Score is {1}".format(get_words(real),get_score(real)))
  print("Fake is {0} and Score is {1}".format(get_words(real),get_score(real)))
  